# **Задание 1 (25 баллов)**
Реализуйте CUDA-программу для вычисления суммы элементов массива с
использованием глобальной памяти. Сравните результат и время выполнения с
последовательной реализацией на CPU для массива размером 100 000 элементов.

In [9]:
%%writefile task1.cu
#include <cuda_runtime.h>                           // подключаем CUDA Runtime API
#include <cstdio>                                   // подключаем printf
#include <vector>                                   // подключаем std::vector
#include <random>                                   // подключаем генератор случайных чисел
#include <chrono>                                   // подключаем средства измерения времени
#include <cstdint>                                  // подключаем типы фиксированной разрядности

#define CHECK_CUDA(call) do {                       /* макрос для проверки CUDA-ошибок */ \
  cudaError_t err = (call);                         /* выполняем вызов CUDA и сохраняем код ошибки */ \
  if (err != cudaSuccess) {                         /* если ошибка не равна cudaSuccess */ \
    printf("CUDA error %s:%d: %s\n",                /* печатаем сообщение об ошибке */ \
           __FILE__, __LINE__, cudaGetErrorString(err)); \
    return 1;                                       /* завершаем программу с ошибкой */ \
  }                                                 /* конец if */ \
} while(0)                                          /* конец макроса */

// --- CUDA kernel: сумма через global memory (atomicAdd) ---
__global__ void sum_global_atomic(const int* a, int n, unsigned long long* out_sum) { // ядро: массив a, размер n, сумма out_sum
  int tid = blockIdx.x * blockDim.x + threadIdx.x;                                   // вычисляем глобальный индекс потока
  int stride = blockDim.x * gridDim.x;                                               // вычисляем шаг сетки (общее число потоков)
  for (int i = tid; i < n; i += stride) {                                            // grid-stride loop: каждый поток суммирует несколько элементов
    atomicAdd(out_sum, (unsigned long long)a[i]);                                    // атомарно добавляем элемент к сумме (global memory)
  }                                                                                   // конец цикла
}                                                                                     // конец ядра

int main() {                                                                          // начало main
  const int N = 100000;                                                               // размер массива (100 000)
  std::vector<int> h_a(N);                                                            // создаем host-массив на CPU
  std::mt19937 rng(123);                                                              // фиксируем seed для воспроизводимости
  std::uniform_int_distribution<int> dist(0, 9);                                      // распределение значений 0..9
  for (int i = 0; i < N; ++i) h_a[i] = dist(rng);                                     // заполняем массив случайными числами

  auto cpu_t0 = std::chrono::high_resolution_clock::now();                            // старт таймера CPU
  long long cpu_sum = 0;                                                              // сумма на CPU (signed long long)
  for (int i = 0; i < N; ++i) cpu_sum += (long long)h_a[i];                           // последовательное суммирование на CPU
  auto cpu_t1 = std::chrono::high_resolution_clock::now();                            // стоп таймера CPU
  double cpu_ms = std::chrono::duration<double, std::milli>(cpu_t1 - cpu_t0).count(); // время CPU в миллисекундах

  int* d_a = nullptr;                                                                 // указатель на массив на GPU
  unsigned long long* d_sum = nullptr;                                                // указатель на сумму на GPU (unsigned long long)
  CHECK_CUDA(cudaMalloc((void**)&d_a, N * sizeof(int)));                              // выделяем память на GPU под массив
  CHECK_CUDA(cudaMalloc((void**)&d_sum, sizeof(unsigned long long)));                 // выделяем память на GPU под сумму
  CHECK_CUDA(cudaMemcpy(d_a, h_a.data(), N * sizeof(int), cudaMemcpyHostToDevice));   // копируем массив CPU->GPU
  CHECK_CUDA(cudaMemset(d_sum, 0, sizeof(unsigned long long)));                       // обнуляем сумму на GPU

  int threads = 256;                                                                  // число потоков в блоке
  int blocks = (N + threads - 1) / threads;                                           // считаем блоки, чтобы покрыть N элементов
  if (blocks > 1024) blocks = 1024;                                                   // ограничиваем число блоков разумным пределом

  cudaEvent_t start, stop;                                                            // объявляем CUDA-события для тайминга
  CHECK_CUDA(cudaEventCreate(&start));                                                // создаем событие start
  CHECK_CUDA(cudaEventCreate(&stop));                                                 // создаем событие stop
  CHECK_CUDA(cudaEventRecord(start));                                                 // записываем старт таймера (в GPU очередь)

  sum_global_atomic<<<blocks, threads>>>(d_a, N, d_sum);                              // запускаем ядро суммирования

  CHECK_CUDA(cudaEventRecord(stop));                                                  // записываем стоп таймера
  CHECK_CUDA(cudaEventSynchronize(stop));                                             // ждем завершения ядра
  CHECK_CUDA(cudaGetLastError());                                                     // проверяем ошибку запуска ядра

  float gpu_ms = 0.0f;                                                                // переменная для времени GPU
  CHECK_CUDA(cudaEventElapsedTime(&gpu_ms, start, stop));                             // получаем время выполнения ядра (мс)

  unsigned long long gpu_sum_u = 0;                                                   // переменная для суммы, пришедшей с GPU
  CHECK_CUDA(cudaMemcpy(&gpu_sum_u, d_sum, sizeof(unsigned long long), cudaMemcpyDeviceToHost)); // копируем сумму GPU->CPU
  long long gpu_sum = (long long)gpu_sum_u;                                           // приводим к signed для сравнения (у нас сумма неотрицательная)

  printf("CPU sum      = %lld\n", cpu_sum);                                           // печатаем сумму CPU
  printf("GPU sum      = %lld\n", gpu_sum);                                           // печатаем сумму GPU
  printf("CPU time     = %.3f ms\n", cpu_ms);                                         // печатаем время CPU
  printf("GPU kernel   = %.3f ms (global memory + atomicAdd)\n", gpu_ms);             // печатаем время ядра GPU

  CHECK_CUDA(cudaEventDestroy(start));                                                // удаляем событие start
  CHECK_CUDA(cudaEventDestroy(stop));                                                 // удаляем событие stop
  CHECK_CUDA(cudaFree(d_a));                                                          // освобождаем память массива на GPU
  CHECK_CUDA(cudaFree(d_sum));                                                        // освобождаем память суммы на GPU
  CHECK_CUDA(cudaDeviceReset());                                                      // сбрасываем устройство (полезно в Colab)

  return 0;                                                                           // завершаем программу успешно
}                                                                                     // конец main

Writing task1.cu


In [10]:
!nvcc -O3 -std=c++17 task1.cu -o task1 \
  -gencode arch=compute_75,code=sm_75
!./task1

CPU sum      = 451583
GPU sum      = 451583
CPU time     = 0.040 ms
GPU kernel   = 0.264 ms (global memory + atomicAdd)


# **Задание 2 (25 баллов)**
Реализуйте CUDA-программу для вычисления префиксной суммы (сканирования)
массива с использованием разделяемой памяти. Сравните время выполнения с
последовательной реализацией на CPU для массива размером 1 000 000 элементов.

In [17]:
%%writefile task2.cu
#include <cuda_runtime.h>                           // подключаем CUDA Runtime API
#include <cstdio>                                   // подключаем printf
#include <vector>                                   // подключаем std::vector
#include <random>                                   // подключаем генератор случайных чисел
#include <chrono>                                   // подключаем измерение времени на CPU
#include <algorithm>                                // подключаем std::min

#define CHECK_CUDA(call) do {                       /* макрос для проверки ошибок CUDA */ \
  cudaError_t err = (call);                         /* выполняем CUDA-вызов */ \
  if (err != cudaSuccess) {                         /* если вернулась ошибка */ \
    printf("CUDA error %s:%d: %s\n",                /* печатаем где и какая ошибка */ \
           __FILE__, __LINE__, cudaGetErrorString(err)); \
    return 1;                                       /* выходим с кодом ошибки */ \
  }                                                 /* конец if */ \
} while(0)                                          /* конец макроса */

// --- Kernel 1: блочный inclusive scan (префиксная сумма) в shared memory + запись суммы блока ---
__global__ void block_inclusive_scan(const int* in, int* out, int n, int* block_sums) { // ядро: вход, выход, размер, суммы блоков
  extern __shared__ int sh[];                                                         // выделяем динамическую shared memory под блок
  int tid = threadIdx.x;                                                              // индекс потока внутри блока
  int gid = blockIdx.x * blockDim.x + tid;                                            // глобальный индекс элемента массива

  int x = 0;                                                                          // переменная для значения элемента (или 0)
  if (gid < n) x = in[gid];                                                           // если индекс в пределах массива — берем значение
  sh[tid] = x;                                                                        // кладем значение в shared memory
  __syncthreads();                                                                    // синхронизируем потоки блока (все должны загрузить sh)

  for (int offset = 1; offset < blockDim.x; offset <<= 1) {                           // цикл по степеням двойки: 1,2,4,8...
    int add = 0;                                                                      // переменная для прибавки
    if (tid >= offset) add = sh[tid - offset];                                        // берем значение слева на offset, если оно существует
    __syncthreads();                                                                  // синхронизация, чтобы все прочитали старые sh
    sh[tid] += add;                                                                   // выполняем шаг Hillis–Steele (inclusive scan)
    __syncthreads();                                                                  // синхронизация, чтобы обновления завершились
  }                                                                                   // конец цикла scan

  if (gid < n) out[gid] = sh[tid];                                                    // записываем результат scan обратно в global memory

  int block_start = blockIdx.x * blockDim.x;                                          // начало диапазона элементов, которые обрабатывает блок
  int valid = min(blockDim.x, n - block_start);                                       // сколько элементов реально есть в блоке (учет хвоста)
  if (tid == valid - 1) block_sums[blockIdx.x] = sh[tid];                             // последний валидный поток записывает сумму блока
}                                                                                     // конец kernel 1

// --- Kernel 2: inclusive scan массива block_sums (ожидаем, что он помещается в один блок) ---
__global__ void scan_block_sums(int* data, int m) {                                   // ядро: делаем inclusive scan для m элементов
  extern __shared__ int sh[];                                                         // shared memory под m значений
  int tid = threadIdx.x;                                                              // индекс потока в блоке

  int x = 0;                                                                          // значение по умолчанию (для потоков вне диапазона)
  if (tid < m) x = data[tid];                                                         // если tid < m — берем соответствующий block sum
  sh[tid] = x;                                                                        // кладем в shared
  __syncthreads();                                                                    // синхронизация

  for (int offset = 1; offset < blockDim.x; offset <<= 1) {                           // такой же Hillis–Steele scan
    int add = 0;                                                                      // прибавка
    if (tid >= offset) add = sh[tid - offset];                                        // берем левый элемент
    __syncthreads();                                                                  // синхронизация перед записью
    sh[tid] += add;                                                                   // обновляем scan
    __syncthreads();                                                                  // синхронизация после записи
  }                                                                                   // конец цикла

  if (tid < m) data[tid] = sh[tid];                                                   // записываем обратно только валидные элементы
}                                                                                     // конец kernel 2

// --- Kernel 3: добавление оффсетов блоков к результату scan каждого блока ---
__global__ void add_block_offsets(int* out, int n, const int* block_prefix, int m) {  // ядро: out + префиксы блоков
  int tid = threadIdx.x;                                                              // индекс потока в блоке
  int gid = blockIdx.x * blockDim.x + tid;                                            // глобальный индекс

  if (gid >= n) return;                                                               // если вышли за пределы — выходим

  int b = blockIdx.x;                                                                 // номер блока текущего элемента
  int offset = 0;                                                                     // оффсет по умолчанию для первого блока
  if (b > 0 && b - 1 < m) offset = block_prefix[b - 1];                               // для блока b берем сумму всех предыдущих блоков

  out[gid] += offset;                                                                 // добавляем оффсет (получаем глобальную префиксную сумму)
}                                                                                     // конец kernel 3

int main() {                                                                          // начало main
  const int N = 1000000;                                                              // размер массива по заданию (1 000 000)
  std::vector<int> h_in(N);                                                           // входной массив на CPU
  std::mt19937 rng(123);                                                              // seed для воспроизводимости
  std::uniform_int_distribution<int> dist(0, 9);                                      // значения 0..9 (без переполнения int на префиксе)
  for (int i = 0; i < N; ++i) h_in[i] = dist(rng);                                    // заполняем вход

  // --- CPU последовательный inclusive prefix sum и его время ---
  std::vector<int> h_cpu(N);                                                          // массив результата на CPU
  auto cpu_t0 = std::chrono::high_resolution_clock::now();                            // старт таймера CPU
  int running = 0;                                                                    // текущая накопленная сумма
  for (int i = 0; i < N; ++i) {                                                       // идем по всем элементам
    running += h_in[i];                                                               // добавляем текущий элемент
    h_cpu[i] = running;                                                               // сохраняем inclusive prefix sum
  }                                                                                   // конец цикла CPU
  auto cpu_t1 = std::chrono::high_resolution_clock::now();                            // стоп таймера CPU
  double cpu_ms = std::chrono::duration<double, std::milli>(cpu_t1 - cpu_t0).count(); // время CPU в мс

  // --- Память на GPU ---
  int* d_in = nullptr;                                                                // указатель на вход на GPU
  int* d_out = nullptr;                                                               // указатель на выход на GPU
  CHECK_CUDA(cudaMalloc((void**)&d_in, N * sizeof(int)));                             // выделяем память под вход
  CHECK_CUDA(cudaMalloc((void**)&d_out, N * sizeof(int)));                            // выделяем память под выход
  CHECK_CUDA(cudaMemcpy(d_in, h_in.data(), N * sizeof(int), cudaMemcpyHostToDevice)); // копируем вход CPU->GPU

  // --- Параметры запуска ---
  int threads = 1024;                                                                 // число потоков в блоке (максимум для многих GPU)
  int blocks = (N + threads - 1) / threads;                                           // число блоков для покрытия N
  int m = blocks;                                                                     // размер массива сумм блоков

  int* d_block_sums = nullptr;                                                        // указатель на суммы блоков
  CHECK_CUDA(cudaMalloc((void**)&d_block_sums, m * sizeof(int)));                     // выделяем память под block_sums

  // --- CUDA events для замера времени всех GPU-ядeр (kernel time total) ---
  cudaEvent_t start, stop;                                                            // события CUDA
  CHECK_CUDA(cudaEventCreate(&start));                                                // создаем start
  CHECK_CUDA(cudaEventCreate(&stop));                                                 // создаем stop
  CHECK_CUDA(cudaEventRecord(start));                                                 // стартуем измерение

  // --- Kernel 1: блочный scan + суммы блоков ---
  block_inclusive_scan<<<blocks, threads, threads * (int)sizeof(int)>>>(              // запускаем kernel 1 с dynamic shared memory
    d_in, d_out, N, d_block_sums);                                                    // передаем аргументы
  CHECK_CUDA(cudaGetLastError());                                                     // проверяем ошибку запуска

  // --- Kernel 2: scan массива block_sums в одном блоке (m обычно < 1024 здесь) ---
  int t2 = 1;                                                                         // переменная для потоков kernel 2
  while (t2 < m) t2 <<= 1;                                                            // округляем число потоков вверх до степени двойки
  if (t2 > 1024) t2 = 1024;                                                           // на всякий случай ограничиваем 1024
  scan_block_sums<<<1, t2, t2 * (int)sizeof(int)>>>(d_block_sums, m);                 // сканируем суммы блоков
  CHECK_CUDA(cudaGetLastError());                                                     // проверяем запуск

  // --- Kernel 3: добавляем оффсеты блоков ко всем элементам (кроме первого блока оффсет > 0) ---
  add_block_offsets<<<blocks, threads>>>(d_out, N, d_block_sums, m);                  // добавляем префиксы блоков
  CHECK_CUDA(cudaGetLastError());                                                     // проверяем запуск

  CHECK_CUDA(cudaEventRecord(stop));                                                  // записываем stop
  CHECK_CUDA(cudaEventSynchronize(stop));                                             // ждем окончания всех GPU работ

  float gpu_ms = 0.0f;                                                                // время GPU в мс
  CHECK_CUDA(cudaEventElapsedTime(&gpu_ms, start, stop));                             // получаем elapsed time (все ядра вместе)

  // --- Скачиваем результат с GPU и проверяем корректность ---
  std::vector<int> h_gpu(N);                                                          // буфер результата на CPU
  CHECK_CUDA(cudaMemcpy(h_gpu.data(), d_out, N * sizeof(int), cudaMemcpyDeviceToHost)); // копируем GPU->CPU

  int ok = 1;                                                                         // флаг корректности
  for (int i = 0; i < N; ++i) {                                                       // проверяем все элементы
    if (h_gpu[i] != h_cpu[i]) {                                                       // если нашли несовпадение
      ok = 0;                                                                         // отмечаем ошибку
      printf("Mismatch at i=%d: CPU=%d GPU=%d\n", i, h_cpu[i], h_gpu[i]);             // выводим где ошибка
      break;                                                                          // выходим из цикла
    }                                                                                 // конец if
  }                                                                                   // конец проверки

  // --- Вывод времени и статуса ---
  printf("CPU time     = %.3f ms (sequential scan)\n", cpu_ms);                       // выводим время CPU
  printf("GPU kernels  = %.3f ms (shared-memory block scan + block offsets)\n", gpu_ms); // выводим время всех GPU-ядeр

  // --- Освобождение ресурсов ---
  CHECK_CUDA(cudaEventDestroy(start));                                                // удаляем событие start
  CHECK_CUDA(cudaEventDestroy(stop));                                                 // удаляем событие stop
  CHECK_CUDA(cudaFree(d_in));                                                         // освобождаем вход на GPU
  CHECK_CUDA(cudaFree(d_out));                                                        // освобождаем выход на GPU
  CHECK_CUDA(cudaFree(d_block_sums));                                                 // освобождаем block sums
  CHECK_CUDA(cudaDeviceReset());                                                      // сброс устройства (полезно в Colab)

  return 0;                                                                           // успешный выход
}                                                                                     // конец main

Overwriting task2.cu


In [18]:
!nvcc -O3 -std=c++17 task2.cu -o task2 \
  -gencode arch=compute_75,code=sm_75
!./task2

task2.cu(143): warning #550-D: variable "ok" was set but never used
    int ok = 1;
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

CPU time     = 0.834 ms (sequential scan)
GPU kernels  = 0.413 ms (shared-memory block scan + block offsets)


# **Задание 3 (25 баллов)**
Реализуйте гибридную программу, в которой обработка массива выполняется
параллельно на CPU и GPU. Первую часть массива обработайте на CPU, вторую — на
GPU. Сравните время выполнения CPU-, GPU- и гибридной реализаций.

In [25]:
%%writefile task3.cu
#include <cuda_runtime.h>                           // CUDA runtime
#include <cstdio>                                   // printf
#include <vector>                                   // vector
#include <random>                                   // rng
#include <chrono>                                   // chrono

#define CHECK_CUDA(call) do {                       /* проверка ошибок CUDA */ \
  cudaError_t err = (call);                         /* вызов */ \
  if (err != cudaSuccess) {                         /* ошибка? */ \
    printf("CUDA error %s:%d: %s\n",                /* печать */ \
           __FILE__, __LINE__, cudaGetErrorString(err)); \
    return 1;                                       /* выход */ \
  }                                                 \
} while(0)

// GPU kernel: сумма части массива (atomicAdd)
__global__ void sum_part_atomic(const int* a, int n, unsigned long long* out_sum) { // ядро
  int tid = blockIdx.x * blockDim.x + threadIdx.x;                                 // id потока
  int stride = blockDim.x * gridDim.x;                                             // шаг
  for (int i = tid; i < n; i += stride) {                                          // цикл по элементам
    atomicAdd(out_sum, (unsigned long long)a[i]);                                  // атомарное сложение
  }                                                                                // конец цикла
}                                                                                  // конец ядра

int main() {                                                                       // main
  const int N = 1'000'000;                                                         // размер массива
  const int HALF = N / 2;                                                          // половина

  std::vector<int> h_a(N);                                                         // массив на CPU
  std::mt19937 rng(123);                                                           // seed
  std::uniform_int_distribution<int> dist(0, 9);                                   // 0..9
  for (int i = 0; i < N; ++i) h_a[i] = dist(rng);                                  // заполнение

  // ---------------- CPU-only ----------------
  auto cpu0 = std::chrono::high_resolution_clock::now();                           // старт CPU-only
  long long cpu_sum = 0;                                                           // сумма CPU
  for (int i = 0; i < N; ++i) cpu_sum += (long long)h_a[i];                        // суммирование
  auto cpu1 = std::chrono::high_resolution_clock::now();                           // стоп CPU-only
  double cpu_ms = std::chrono::duration<double, std::milli>(cpu1 - cpu0).count();  // время CPU-only

  // ---------------- GPU memory ----------------
  int* d_a = nullptr;                                                              // массив на GPU
  unsigned long long* d_sum = nullptr;                                             // сумма на GPU
  CHECK_CUDA(cudaMalloc(&d_a, N * sizeof(int)));                                   // malloc массива
  CHECK_CUDA(cudaMalloc(&d_sum, sizeof(unsigned long long)));                      // malloc суммы
  CHECK_CUDA(cudaMemcpy(d_a, h_a.data(), N * sizeof(int), cudaMemcpyHostToDevice)); // копирование H2D

  // ---------------- GPU-only (kernel time) ----------------
  CHECK_CUDA(cudaMemset(d_sum, 0, sizeof(unsigned long long)));                    // обнуление
  cudaEvent_t g0, g1;                                                              // события
  CHECK_CUDA(cudaEventCreate(&g0));                                                // create
  CHECK_CUDA(cudaEventCreate(&g1));                                                // create
  CHECK_CUDA(cudaEventRecord(g0));                                                 // старт
  sum_part_atomic<<<256, 256>>>(d_a, N, d_sum);                                    // kernel
  CHECK_CUDA(cudaEventRecord(g1));                                                 // стоп
  CHECK_CUDA(cudaEventSynchronize(g1));                                            // ждать
  float gpu_ms = 0.0f;                                                             // время
  CHECK_CUDA(cudaEventElapsedTime(&gpu_ms, g0, g1));                               // elapsed
  unsigned long long gpu_sum_u = 0;                                                // сумма GPU
  CHECK_CUDA(cudaMemcpy(&gpu_sum_u, d_sum, sizeof(unsigned long long), cudaMemcpyDeviceToHost)); // D2H
  long long gpu_sum = (long long)gpu_sum_u;                                        // cast

  // ---------------- Hybrid PARALLEL with separate timings ----------------
  CHECK_CUDA(cudaMemset(d_sum, 0, sizeof(unsigned long long)));                    // обнуление для hybrid

  cudaEvent_t hg0, hg1;                                                            // события для GPU части в hybrid
  CHECK_CUDA(cudaEventCreate(&hg0));                                               // create
  CHECK_CUDA(cudaEventCreate(&hg1));                                               // create

  auto hyb0 = std::chrono::high_resolution_clock::now();                           // старт общего hybrid (wall time)

  CHECK_CUDA(cudaEventRecord(hg0));                                                // старт kernel таймера hybrid-GPU
  sum_part_atomic<<<256, 256>>>(d_a + HALF, HALF, d_sum);                          // GPU обрабатывает 2-ю половину (асинхронно)
  CHECK_CUDA(cudaEventRecord(hg1));                                                // стоп kernel таймера hybrid-GPU (в той же очереди)

  auto hcpu0 = std::chrono::high_resolution_clock::now();                          // старт таймера hybrid-CPU
  long long cpu_part = 0;                                                          // сумма CPU части
  for (int i = 0; i < HALF; ++i) cpu_part += (long long)h_a[i];                    // CPU считает 1-ю половину параллельно GPU
  auto hcpu1 = std::chrono::high_resolution_clock::now();                          // стоп таймера hybrid-CPU
  double hybrid_cpu_ms = std::chrono::duration<double, std::milli>(hcpu1 - hcpu0).count(); // время CPU-части

  CHECK_CUDA(cudaEventSynchronize(hg1));                                           // ждём завершения GPU kernel в hybrid
  float hybrid_gpu_kernel_ms = 0.0f;                                               // время kernel в hybrid
  CHECK_CUDA(cudaEventElapsedTime(&hybrid_gpu_kernel_ms, hg0, hg1));               // elapsed kernel hybrid-GPU

  unsigned long long gpu_part_u = 0;                                               // сумма GPU части
  CHECK_CUDA(cudaMemcpy(&gpu_part_u, d_sum, sizeof(unsigned long long), cudaMemcpyDeviceToHost)); // D2H
  long long gpu_part = (long long)gpu_part_u;                                      // cast

  long long hybrid_sum = cpu_part + gpu_part;                                      // итоговая сумма hybrid

  auto hyb1 = std::chrono::high_resolution_clock::now();                           // стоп общего hybrid
  double hybrid_total_ms = std::chrono::duration<double, std::milli>(hyb1 - hyb0).count(); // общее время hybrid

  // ---------------- Print ----------------
  printf("CPU only sum        = %lld, time = %.3f ms\n", cpu_sum, cpu_ms);          // CPU-only
  printf("GPU only sum        = %lld, time = %.3f ms (kernel)\n", gpu_sum, gpu_ms); // GPU-only kernel
  printf("Hybrid CPU part     = %lld, time = %.3f ms\n", cpu_part, hybrid_cpu_ms);  // hybrid CPU часть + время
  printf("Hybrid GPU part     = %lld, time = %.3f ms (kernel)\n", gpu_part, hybrid_gpu_kernel_ms); // hybrid GPU часть + kernel time
  printf("Hybrid total sum    = %lld, time = %.3f ms (wall)\n", hybrid_sum, hybrid_total_ms); // hybrid total wall time

  // ---------------- Cleanup ----------------
  CHECK_CUDA(cudaEventDestroy(g0));                                                // destroy
  CHECK_CUDA(cudaEventDestroy(g1));                                                // destroy
  CHECK_CUDA(cudaEventDestroy(hg0));                                               // destroy
  CHECK_CUDA(cudaEventDestroy(hg1));                                               // destroy
  CHECK_CUDA(cudaFree(d_a));                                                       // free
  CHECK_CUDA(cudaFree(d_sum));                                                     // free
  CHECK_CUDA(cudaDeviceReset());                                                   // reset

  return 0;                                                                        // exit
}

Overwriting task3.cu


In [26]:
!nvcc -O3 -std=c++17 task3.cu -o task3 \
  -gencode arch=compute_75,code=sm_75
!./task3

CPU only sum        = 4505398, time = 0.293 ms
GPU only sum        = 4505398, time = 1.509 ms (kernel)
Hybrid CPU part     = 2254182, time = 0.354 ms
Hybrid GPU part     = 2251216, time = 0.741 ms (kernel)
Hybrid total sum    = 4505398, time = 0.773 ms (wall)


# **Задание 4 (25 баллов)**
Реализуйте распределённую программу с использованием MPI для обработки массива
данных. Разделите массив между процессами, выполните вычисления локально и
соберите результаты. Проведите замеры времени выполнения для 2, 4 и 8 процессов.

In [27]:
!apt-get -qq update
!apt-get -qq install -y mpich

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package libslurm37.
(Reading database ... 121689 files and directories currently installed.)
Preparing to unpack .../libslurm37_21.08.5-2ubuntu1_amd64.deb ...
Unpacking libslurm37 (21.08.5-2ubuntu1) ...
Selecting previously unselected package hwloc-nox.
Preparing to unpack .../hwloc-nox_2.7.0-2ubuntu1_amd64.deb ...
Unpacking hwloc-nox (2.7.0-2ubuntu1) ...
Selecting previously unselected package libmpich12:amd64.
Preparing to unpack .../libmpich12_4.0-3_amd64.deb ...
Unpacking libmpich12:amd64 (4.0-3) ...
Selecting previously unselected package mpich.
Preparing to unpack .../archives/mpich_4.0-3_amd64.deb ...
Unpacking mpich (4.0-3) ...
Selecting previously unselected package libmpich-dev:amd64.
Preparing to unpack .../libmpich-dev_4.0-3_amd64.deb ...
Unpacking libmpich

In [28]:
%%writefile task4.cpp
#include <mpi.h>                                      // подключаем библиотеку MPI
#include <cstdio>                                     // подключаем printf
#include <vector>                                     // подключаем std::vector
#include <random>                                     // подключаем генератор случайных чисел
#include <cstdint>                                    // подключаем целочисленные типы
#include <algorithm>                                  // подключаем std::min

int main(int argc, char** argv) {                     // главная функция программы (argc/argv нужны MPI)
  MPI_Init(&argc, &argv);                             // инициализируем MPI-среду

  int rank = 0;                                       // переменная для номера процесса
  int size = 0;                                       // переменная для количества процессов
  MPI_Comm_rank(MPI_COMM_WORLD, &rank);               // получаем номер текущего процесса (rank)
  MPI_Comm_size(MPI_COMM_WORLD, &size);               // получаем общее число процессов (size)

  const int N = 10'000'000;                           // общий размер массива (можно менять для экспериментов)
  std::vector<int> sendcounts(size);                  // массив: сколько элементов отправить каждому процессу
  std::vector<int> displs(size);                      // массив: смещения (offset) для Scatterv

  int base = N / size;                                // базовый размер куска на процесс
  int rem  = N % size;                                // остаток, который распределим по первым процессам

  for (int p = 0; p < size; ++p) {                    // цикл по всем процессам
    sendcounts[p] = base + (p < rem ? 1 : 0);         // первым rem процессам даём на 1 элемент больше
  }                                                   // конец цикла

  displs[0] = 0;                                      // смещение для первого процесса = 0
  for (int p = 1; p < size; ++p) {                    // цикл для вычисления смещений
    displs[p] = displs[p - 1] + sendcounts[p - 1];    // смещение = сумма всех предыдущих sendcounts
  }                                                   // конец цикла

  std::vector<int> global;                            // глобальный массив (будет только на rank 0)
  if (rank == 0) {                                    // если это главный процесс
    global.resize(N);                                 // выделяем память под весь массив
    std::mt19937 rng(123);                            // фиксируем seed для воспроизводимости
    std::uniform_int_distribution<int> dist(0, 9);    // распределение значений 0..9
    for (int i = 0; i < N; ++i) global[i] = dist(rng);// заполняем массив случайными числами
  }                                                   // конец if (rank==0)

  int local_n = sendcounts[rank];                     // сколько элементов получит текущий процесс
  std::vector<int> local(local_n);                    // локальный массив для текущего процесса

  MPI_Barrier(MPI_COMM_WORLD);                        // барьер: все процессы стартуют измерение синхронно
  double t0 = MPI_Wtime();                            // старт таймера MPI (секунды)

  MPI_Scatterv(                                       // распределяем части массива по процессам
    rank == 0 ? global.data() : nullptr,              // буфер отправки (только у rank 0, иначе nullptr)
    sendcounts.data(),                                // сколько отправлять каждому процессу
    displs.data(),                                    // смещения в глобальном массиве
    MPI_INT,                                          // тип элементов отправки (int)
    local.data(),                                     // буфер приёма локальных данных
    local_n,                                          // сколько принять (для текущего процесса)
    MPI_INT,                                          // тип элементов приёма
    0,                                                // корневой процесс (root) = 0
    MPI_COMM_WORLD                                    // коммуникатор всех процессов
  );                                                  // конец Scatterv

  long long local_sum = 0;                            // локальная сумма на процессе
  for (int i = 0; i < local_n; ++i) {                 // цикл по локальному куску
    local_sum += (long long)local[i];                 // суммируем локальные элементы
  }                                                   // конец цикла

  long long global_sum = 0;                           // переменная для глобальной суммы (будет значима на rank 0)
  MPI_Reduce(                                         // собираем локальные суммы в одну глобальную
    &local_sum,                                       // адрес локальной суммы
    &global_sum,                                      // адрес глобальной суммы (на root)
    1,                                                // число элементов (одна сумма)
    MPI_LONG_LONG,                                    // тип данных (long long)
    MPI_SUM,                                          // операция редукции (сумма)
    0,                                                // root = 0
    MPI_COMM_WORLD                                    // коммуникатор
  );                                                  // конец Reduce

  MPI_Barrier(MPI_COMM_WORLD);                        // барьер: ждём, чтобы все завершили вычисления
  double t1 = MPI_Wtime();                            // стоп таймера MPI

  if (rank == 0) {                                    // если главный процесс
    double ms = (t1 - t0) * 1000.0;                   // переводим секунды в миллисекунды
    printf("MPI processes: %d | N=%d | sum=%lld | time=%.3f ms\n", size, N, global_sum, ms); // печать результатов
  }                                                   // конец if (rank==0)

  MPI_Finalize();                                     // завершаем работу MPI
  return 0;                                           // выход из программы
}                                                     // конец main

Writing task4.cpp


In [32]:
!mpicxx -O3 task4.cpp -o task4

In [37]:
!mpirun --allow-run-as-root --use-hwthread-cpus --oversubscribe -np 2 ./task4
!mpirun --allow-run-as-root --use-hwthread-cpus --oversubscribe -np 4 ./task4
!mpirun --allow-run-as-root --use-hwthread-cpus --oversubscribe -np 8 ./task4

MPI processes: 2 | N=10000000 | sum=45004663 | time=11.065 ms
MPI processes: 4 | N=10000000 | sum=45004663 | time=12.384 ms
MPI processes: 8 | N=10000000 | sum=45004663 | time=17.198 ms
